In [11]:
import psycopg2
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import nltk.tokenize as tk
import nltk
import string
import gensim
from gensim import corpora, models, similarities
from langdetect import DetectorFactory
from langdetect import detect
from datetime import datetime

DetectorFactory.seed = 0

In [25]:
## Cleaning and Preprocessing

stop = set(stopwords.words('english'))
additional_exclude_words = [u'beer', u'one', u'like', u'well', u'really', u'very', u'it\x92s', 
                            u'it', u'beers',u'would',u'taste',u'tastes',u'get',u'i\x92m',u'quite', u'i\x92ve',
                            u'bit',u'much',u'good',u'better',u'think',u'first',u'new',u'try', u'updated']
stop |= set(additional_exclude_words)
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc, name_tokens):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = " ".join([ch for ch in doc.lower().split() if ch not in exclude])
    normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
    tokens = tk.word_tokenize(normalized)
    tokens_with_pos = [x[0] for x in nltk.pos_tag(tokens) if x[1] in ('NN','JJ') and 
                       x[0] not in name_tokens and len(x[0])>2 and x[0] not in stop]
    return tokens_with_pos

In [26]:
## Establishing Connection

creds = {'user': 'rthakur',
         'password': 'brew',
         'host': 'capstone-brewing.cyol5m7lekm6.us-east-1.rds.amazonaws.com',
         'dbname': 'brew'}
try:
    conn = psycopg2.connect("dbname={} user={} password={} host={}".format(creds['dbname'], creds['user'], 
                                                                           creds['password'], creds['host']))
except:
    print "Unable to connect to the database"

In [15]:
cur = conn.cursor()
cur.execute("""
    create table wrk.rohan_test AS (select * from wrk.beer limit 50)""")

In [16]:
print cur.fetchall()

ProgrammingError: no results to fetch

In [27]:
print str(datetime.now())

cur = conn.cursor()
cur.execute("""
    select b.id,
       b.name,
       r.text
    from wrk.beer b left join wrk.review r on b.id = r.beer_id
    """)
rows = cur.fetchall()
reviews_clean = []
review_ids = []

for row in rows:
    if not row[2]:
        continue
    review = row[2].decode('utf-8')
    name = row[1]
    name_tokens = nltk.word_tokenize(name.lower())
    try:
        if detect(review)=='en':
            rev_clean = clean(review.lower(), name_tokens)
            reviews_clean.append(rev_clean)
            review_ids.append(int(row[0]))
    except:
        continue
dictionary = corpora.Dictionary(reviews_clean)

print str(datetime.now())

2017-08-12 18:30:18.313000
2017-08-12 23:17:52.105000


In [8]:
print reviews_clean

depth


In [28]:
dictionary.save('../../data/beerwords_v5.dict')

In [12]:
dictionary = corpora.Dictionary.load('../../data/beerwords_v5.dict')

In [29]:
cur = conn.cursor()
corpus = []
corpus_indices = []
revs_ids = zip(review_ids, reviews_clean)
for x in range(4611):
    if x%500==0:
        print x
        print str(datetime.now())
    if x+1 not in review_ids:
        continue
    current_revs = [r[1] for r in revs_ids if r[0]==x+1]
    revs_for_corpus = [item for sublist in current_revs for item in sublist]
    #dict_temp = corpora.Dictionary(current_revs)
    #dict_temp.save('../../data/dictionaries/dict_' + str(x) + '.dict')
    corpus += [dictionary.doc2bow(revs_for_corpus)]
    corpus_indices.append(x+1)
    
corpora.MmCorpus.serialize('../../data/beercorpus_v5.mm', corpus)

0
2017-08-13 00:37:43.479000
500
2017-08-13 00:40:25.655000
1000
2017-08-13 00:43:36.273000
1500
2017-08-13 00:46:42.996000
2000
2017-08-13 00:49:19.020000
2500
2017-08-13 00:52:15.805000
3000
2017-08-13 00:54:36.842000
3500
2017-08-13 00:57:25.332000
4000
2017-08-13 00:59:28.665000
4500
2017-08-13 01:02:00.518000


In [13]:
corpus = corpora.MmCorpus('../../data/beercorpus_v5.mm')
beer_model = models.LdaModel.load('../../data/beer_model_v5', mmap='r')
#print str(datetime.now())
#beer_model =  models.LdaModel(corpus, id2word=dictionary, num_topics=200)
#print str(datetime.now())

#beer_model.save('../../data/beer_model_v5')

IOError: [Errno 2] No such file or directory: '../../data/beer_model_v5.expElogbeta.npy'

In [31]:
corpus_indices = []
for x in range(4611):
    if x%500==0:
        print x
        print str(datetime.now())
    if x+1 not in review_ids:
        continue
    corpus_indices.append(x+1)

0
2017-08-13 01:30:27.697000
500
2017-08-13 01:30:28.591000
1000
2017-08-13 01:30:29.777000
1500
2017-08-13 01:30:31.297000
2000
2017-08-13 01:30:33.187000
2500
2017-08-13 01:30:35.495000
3000
2017-08-13 01:30:37.222000
3500
2017-08-13 01:30:40.386000
4000
2017-08-13 01:30:43.536000
4500
2017-08-13 01:30:44.996000


In [32]:
#Similarity
print str(datetime.now())
index = similarities.MatrixSimilarity(beer_model[corpus])
print str(datetime.now())

2017-08-13 01:31:32.502000
2017-08-13 01:36:04.399000


In [33]:
#Similarity
print str(datetime.now())
df_dict = {'beer_id':[], 'similar_beer_id':[], 'similarity_score':[]}
for i, corp in zip(corpus_indices, corpus):
    b = beer_model[corp]
    like_kernel = sorted(list(enumerate(index[b])), key = lambda x:x[1], reverse=True)[1:]
    df_dict['beer_id'].extend([i]*len(like_kernel))
    df_dict['similar_beer_id'].extend([tup[0] for tup in like_kernel])
    df_dict['similarity_score'].extend([tup[1] for tup in like_kernel])
print str(datetime.now())

2017-08-13 01:39:37.427000
2017-08-13 01:42:33.096000


In [34]:
#create dataframe
import pandas as pd
df = pd.DataFrame.from_dict(data=df_dict)

In [35]:
df.to_csv(path_or_buf = '../../data/beer_similarities_complete.csv')

In [38]:
print dictionary[572]

creaminess


In [10]:
print 'At the very beginning: ' + str(datetime.now())
import pandas as pd
from gensim import corpora, models, similarities

print 'Imports complete: ' + str(datetime.now())

dictionary = corpora.Dictionary.load('../../data/beerwords_v5.dict')
corpus = corpora.MmCorpus('../../data/beercorpus_v5.mm')
corpus_indices = pd.read_csv('../../data/correct_corpus_ids.csv')['corpus_ids'].tolist()

print 'Loaded all files ' + str(datetime.now())

def get_cloud_inputs(beer_id, dictionary, corpus, corpus_indices, num_words=200):
    '''
    beer_id: ID of the beer (from wrk.beer) for which you want the word cloud.
    dictionary: type gensim.corpora.dictionary -> load this from saved files
    corpus: type gensim.corpora.MMCorpus -> load this from saved files
    corpus_indices: Mapping list for corpus to beer
    num_words: Number of highest frequency words to be returned 
    '''
    for i, corp in zip(corpus_indices, corpus):
        if i==beer_id:
            correct_corp = corp
            break
    sorted_corpus =  sorted(correct_corp, key = lambda x:x[1], reverse=True)[:num_words]
    return [(dictionary[id_], count_) for id_, count_ in sorted_corpus]

get_cloud_inputs(2345, dictionary, corpus, corpus_indices)

print 'Actual function done: ' + str(datetime.now())


At the very beginning: 2017-08-13 12:07:53.416000
Imports complete: 2017-08-13 12:07:53.417000
Loaded all files 2017-08-13 12:07:53.745000
Actual function done: 2017-08-13 12:08:24.565000


In [46]:
df_corpus_ids = pd.DataFrame(corpus_indices, columns=["corpus_ids"])
df_corpus_ids.to_csv(path_or_buf = '../../data/correct_corpus_ids.csv', index=False)